In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# sns.set_style('whitegrid')

df_orig_breed = pd.read_csv('data/breed_labels.csv')
df_orig_color = pd.read_csv('data/color_labels.csv')
df_orig_states = pd.read_csv('data/state_labels.csv')

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

print(df_orig_breed.head())
print(df_orig_color)
print(df_orig_states)

print('\n')
print(df_train.describe())
print('\n')
print(df_train.info())
print('\n')
print(df_train.columns)
# df_train.head()
print(df_test.head())


   BreedID  Type         BreedName
0        1     1     Affenpinscher
1        2     1      Afghan Hound
2        3     1  Airedale Terrier
3        4     1            Akbash
4        5     1             Akita
   ColorID ColorName
0        1     Black
1        2     Brown
2        3    Golden
3        4    Yellow
4        5     Cream
5        6      Gray
6        7     White
    StateID        StateName
0     41336            Johor
1     41325            Kedah
2     41367         Kelantan
3     41401     Kuala Lumpur
4     41415           Labuan
5     41324           Melaka
6     41332  Negeri Sembilan
7     41335           Pahang
8     41330            Perak
9     41380           Perlis
10    41327     Pulau Pinang
11    41345            Sabah
12    41342          Sarawak
13    41326         Selangor
14    41361       Terengganu


               Type           Age        Breed1        Breed2        Gender  \
count  14993.000000  14993.000000  14993.000000  14993.000000  14993.000000  

# Load Data

In [50]:
# X_column = df_train.columns['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
#        'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
#        'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'PetID']

print(df_train[['State', 'PetID']].head())
y_column = df_train['AdoptionSpeed']
print(len(y_column))
# print(X_column)
# print(y_column)
# print(df_train['Color1'].sort_values().unique())
# print(df_train['Color2'].sort_values().unique())
# print(df_train['Color3'].sort_values().unique())

df_train[df_train.isnull().any(axis=1)].head()
# print(df_train['AdoptionSpeed'] == 0)
# print('')

   State      PetID
0  41326  86e1089a3
1  41401  6296e909a
2  41326  3422e4906
3  41401  5842f1ff5
4  41326  850a43f90
14993


,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
5,2,NaN,3,266,0,2,5,6,0,2,...,1,1,0,41326,22fe332bf9c924d4718005891c63fbed,0,This is a stray kitten that came to my house. ...,d24c30b4b,2.0,2
8,2,NaN,2,265,0,2,6,0,0,2,...,1,1,0,41326,d8af7afece71334473575c9f70daf00d,0,"healthy and active, feisty kitten found in nei...",c06d167ca,6.0,1
19,2,NaN,1,266,0,3,1,2,7,1,...,1,5,0,41326,b752f78276215f44581eeb6eea1e63bc,0,"Birth Date: Oct 30th Kitty 1, Melly -Female Sa...",b10e7605a,1.0,4
33,2,NaN,4,266,0,2,1,6,7,1,...,1,2,0,41326,7d58438884ab468dce87c7e252bbd6e4,0,Two gorgeous kittens have just lost their mumm...,9415bc79e,7.0,3
36,2,NaN,24,265,292,2,1,4,0,2,...,1,3,0,41326,90b00f90ffdf9ec1cac529a2bbef3ecc,0,she fat n healthy. in door cat,61fa73996,2.0,4


In [3]:
df_train.dtypes

Type               int64
Name              object
Age                int64
Breed1             int64
Breed2             int64
Gender             int64
Color1             int64
Color2             int64
Color3             int64
MaturitySize       int64
FurLength          int64
Vaccinated         int64
Dewormed           int64
Sterilized         int64
Health             int64
Quantity           int64
Fee                int64
State              int64
RescuerID         object
VideoAmt           int64
Description       object
PetID             object
PhotoAmt         float64
AdoptionSpeed      int64
dtype: object

# Feature Engineering

In [4]:
# select the columns
X_columns = [x for x in df_train.columns if x != 'AdoptionSpeed' and df_train.loc[:,x].dtype != object]
y_column = ['AdoptionSpeed']
list(X_columns)

['Type',
 'Age',
 'Breed1',
 'Breed2',
 'Gender',
 'Color1',
 'Color2',
 'Color3',
 'MaturitySize',
 'FurLength',
 'Vaccinated',
 'Dewormed',
 'Sterilized',
 'Health',
 'Quantity',
 'Fee',
 'State',
 'VideoAmt',
 'PhotoAmt']

# Model Training

## Linear Regression

In [5]:
threshold = 0.8
X = df_train[X_columns]
y = df_train[y_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1.0-threshold, shuffle=True)

print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

X_train (11994, 19)
y_train (11994, 1)
X_test (2999, 19)
y_test (2999, 1)


In [6]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [8]:
print(y_pred)

[[2.43962902]
 [2.62152609]
 [2.27740581]
 ...
 [2.24696369]
 [2.63352222]
 [2.83057707]]


# Experiments

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

def model_training(model_name, model, X_train, y_train):
    model.fit(X_train, y_train)
    return model
    
def model_prediction(model, X_test):
    y_pred = model.predict(X_test)
    return y_pred

def model_evaluation(model_name, y_test, y_pred):
    print(model_name)
    print('MAE', mean_absolute_error(y_test, y_pred))
    print('RMSE', np.sqrt(mean_squared_error(y_test, y_pred)))
    plt.scatter(y_test, y_pred, alpha=0.3)
    plt.plot(range(0,30000, 100), range(0,30000, 100), '--r', alpha=0.3, label='Line1')
    plt.title(model_name)
    plt.xlabel('True Value')
    plt.ylabel('Predict Value')
    plt.xlim([0, 30000])
    plt.ylim([0, 30000])
    plt.show()
    print('')

# plt.cla()
# plt.clf()
# plt.close()

    
def run_experiment(model_name, model, X_train, y_train, X_test):
    train_model = model_training(model_name, model, X_train, y_train)
    predictions = model_prediction(train_model, X_test)
    model_evaluation(model_name, y_test, predictions)
    
run_experiment('Linear Regression', LinearRegression(), X_train, y_train, X_test)
run_experiment('KNN 5', KNeighborsRegressor(5), X_train, y_train, X_test)
run_experiment('KNN 2', KNeighborsRegressor(2), X_train, y_train, X_test)
run_experiment('Decision Tree', DecisionTreeRegressor(), X_train, y_train, X_test)
run_experiment('Random Forest 10', RandomForestRegressor(10), X_train, y_train, X_test)
run_experiment('Random Forest 100', RandomForestRegressor(100), X_train, y_train, X_test)


Linear Regression
MAE 0.9831860271724969
RMSE 1.141610176612629


NameError: name 'plt' is not defined